# Index

## [Libraries](#1.-Import-the-Libraries)
## [Analysis](#2.-Preliminary-Analysis)
## [Preprocessing]

# 🚌 Bus Demand Forecasting Hackathon

## 📌 Problem Statement

Accurately forecasting demand for bus journeys is a complex task due to the influence of multiple factors. These include:

- **Holiday calendars**
- **Wedding seasons**
- **Long weekends**
- **School vacations**
- **Exam schedules**
- **Day-of-week effects**
- **Regional holidays** (impact varies by region)

> Not all holidays cause significant changes in demand, making this a non-trivial modeling problem.

---

## 🎯 Objective

Develop a predictive model that forecasts the **total number of seats booked** for a given **route** and **date of journey**, exactly **15 days before the travel date**.

---

## 📂 Provided Data

You will be given historical data from the platform, which includes:

- `seats_booked`: Number of seats booked (actual demand)
- `date_of_journey`: The actual travel date
- `date_of_issue`: The date when the ticket was booked
- `search_data`: Number of user searches for a specific journey on a given booking date

---

## 🧠 Challenge Details

You need to predict the **final demand (total seats booked)** for each route, **15 days in advance** of the travel date.

### ✅ Example

- **Route**: Source City "A" → Destination City "B"
- **Date of Journey (DOJ)**: `30-Jan-2025`
- **Prediction Date**: `16-Jan-2025` (15 days prior to DOJ)

Your model should predict the **total expected bookings** on the prediction date for the journey date.

---

## 📥 Data Access

Download the training and testing datasets from the links provided at the bottom of the problem statement.

---

## 🔧 Next Steps

1. Exploratory Data Analysis (EDA)
2. Feature Engineering
3. Model Building
4. Evaluation and Validation
5. Final Predictions



# Data

## Data Collection

Data - https://www.analyticsvidhya.com/datahack/contest/redbus-data-decode-hackathon-2025/
1. train_csv => rows - 67200, columns - 4
  - ['doj', 'srcid', 'destid', 'final_seatcount']
2. test_csv  => rows - 5900, columns - 4
  - ['route_key', 'doj', 'srcid', 'destid']
3. transactions.csv => rows - 22661000, columns - 11
  - ['doj', 'doi', 'srcid', 'destid', 'srcid_region', 'destid_region',
       'srcid_tier', 'destid_tier', 'cumsum_seatcount', 'cumsum_searchcount',
       'dbd']

## Import the required Libraries

In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

## Load the Data

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
transactions_data = pd.read_csv("transactions.csv")

In [3]:
train_df = train_data.copy()
test_df = test_data.copy()
test_df = test_df.drop('route_key', axis = 1)
trans_df = transactions_data.copy()

### Holidays 

In [4]:
holiday_df = pd.read_csv('indian_holidays_2023_2025.csv')
holiday_df = holiday_df[holiday_df['type'] == 'Restricted Holiday']
# print(holiday_df.shape) # (112, 9)
holiday_df = holiday_df.drop(['type', 'description','country', 'created_at', 'updated_at'], axis = 1)
# print(holiday_df.shape) # (112, 4)
holiday_df = holiday_df.rename({'date': 'doh'}, axis = 1)
# holiday_df['doh'] = pd.to_datetime(holiday_df['doh'], format = '%Y-%m-%d')

## Understand the Data

### Shape of each DataFrame

In [5]:
print("train shape ->", train_df.shape)
print("transactions shape ->", trans_df.shape)
print("test shape ->", test_df.shape)
print("Holidays data ->", holiday_df.shape)

train shape -> (67200, 4)
transactions shape -> (2266100, 11)
test shape -> (5900, 3)
Holidays data -> (112, 4)


In [6]:
############ Target Range ################
print('Minimum Seat count -->', min(train_df['final_seatcount']))
print('Maximum Seat count -->', max(train_df['final_seatcount']))

Minimum Seat count --> 2.0
Maximum Seat count --> 13503.0


In [7]:
############ Data Range ####################
trans_doi = pd.to_datetime(trans_df['doi'], format = '%Y-%m-%d')
print(trans_doi.min().date())
trans_doj = pd.to_datetime(trans_df['doj'], format = '%Y-%m-%d')
print(trans_doj.max().date())

2023-01-30
2025-02-28


### .head() 

In [8]:
train_df.head()

,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0
3,2023-03-01,47,45,2580.0
4,2023-03-01,46,9,4185.0


In [9]:
test_df.head()

,doj,srcid,destid
0,2025-02-11,46,45
1,2025-01-20,17,23
2,2025-01-08,2,14
3,2025-01-08,8,47
4,2025-01-08,9,46


In [10]:
trans_df.head()

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30
3,2023-03-01,2023-01-30,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,0.0,68.0,30
4,2023-03-01,2023-01-30,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,9.0,162.0,30


In [11]:
holiday_df.head()

,holiday_id,name,doh,occasion_id
0,6952,New Year's Day,2023-01-01,7017
1,6953,Makar Sankranti,2023-01-14,7018
3,6955,Pongal,2023-01-15,7020
6,6958,Vasant Panchami,2023-01-26,7023
7,6959,Guru Ravidas Jayanti,2023-02-05,7024


### .info()

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doj              67200 non-null  object 
 1   srcid            67200 non-null  int64  
 2   destid           67200 non-null  int64  
 3   final_seatcount  67200 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.1+ MB


In [13]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5900 entries, 0 to 5899
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doj     5900 non-null   object
 1   srcid   5900 non-null   int64 
 2   destid  5900 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 138.4+ KB


In [14]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2266100 entries, 0 to 2266099
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   doj                 object 
 1   doi                 object 
 2   srcid               int64  
 3   destid              int64  
 4   srcid_region        object 
 5   destid_region       object 
 6   srcid_tier          object 
 7   destid_tier         object 
 8   cumsum_seatcount    float64
 9   cumsum_searchcount  float64
 10  dbd                 int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 190.2+ MB


In [15]:
holiday_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 229
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   holiday_id   112 non-null    int64 
 1   name         112 non-null    object
 2   doh          112 non-null    object
 3   occasion_id  112 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 4.4+ KB


### .describe()

In [16]:
train_df.describe()

,srcid,destid,final_seatcount
count,67200.000000,67200.000000,67200.000000
mean,29.550000,29.750000,2001.729464
std,16.670082,15.938986,1194.711140
min,1.000000,1.000000,2.000000
25%,14.750000,15.750000,1252.000000
50%,36.000000,34.500000,1685.000000
75%,45.000000,45.000000,2408.000000
max,48.000000,48.000000,13503.000000


In [17]:
test_df.describe()

,srcid,destid
count,5900.000000,5900.000000
mean,29.550000,29.750000
std,16.671371,15.940219
min,1.000000,1.000000
25%,14.750000,15.750000
50%,36.000000,34.500000
75%,45.000000,45.000000
max,48.000000,48.000000


In [18]:
round(trans_df.describe(), 2)

,srcid,destid,cumsum_seatcount,cumsum_searchcount,dbd
count,2266100.00,2266100.00,2266100.00,2266100.00,2266100.00
mean,29.55,29.75,173.84,4742.16,15.00
std,16.67,15.94,485.89,16735.50,8.94
min,1.00,1.00,0.00,0.00,0.00
25%,14.75,15.75,0.00,208.00,7.00
50%,36.00,34.50,16.00,764.00,15.00
75%,45.00,45.00,100.00,2896.00,23.00
max,48.00,48.00,13503.00,1677297.00,30.00


In [19]:
holiday_df.describe()

,holiday_id,occasion_id
count,112.000000,112.000000
mean,9523.535714,7065.098214
std,3353.871424,97.883177
min,6952.000000,7017.000000
25%,7011.750000,7030.000000
50%,7271.500000,7051.000000
75%,14160.500000,7075.500000
max,14219.000000,7767.000000


## transform dtype of date columns

In [21]:
# object into datetime
dfs = [train_df, test_df, trans_df, holiday_df]
date_cols = ['doj', 'doi', 'doh']

for df in dfs:
    for col in date_cols:
        if col in df.columns and df[col].dtype != 'datetime64[ns]':
            df[col] = pd.to_datetime(df[col])

### Check for date columns dtype

In [25]:
print('train_df ->',train_df['doj'].dtype)
print('test_df ->',test_df['doj'].dtype)
print('trans_df - doj ->',trans_df['doj'].dtype)
print('trans_df - doi ->',trans_df['doj'].dtype)
print('holiday_df ->',train_df['doj'].dtype)

train_df -> datetime64[ns]
test_df -> datetime64[ns]
trans_df - doj -> datetime64[ns]
trans_df - doi -> datetime64[ns]
holiday_df -> datetime64[ns]


### 2.2 Merge data from transactions file to test and train files

In [ ]:
transaction_15 = trans_df[trans_df['dbd'] == 15]

features = transaction_15[['doj', 'srcid', 'destid', 'srcid_region', 'destid_region',
       'srcid_tier', 'destid_tier', 'cumsum_seatcount', 'cumsum_searchcount']]

train_df = train_df.merge(features, on = ['doj', 'srcid', 'destid'], how = 'left')
test_df = test_df.merge(features, on = ['doj', 'srcid', 'destid'], how = 'left')

In [ ]:
train_df['srcid_region'].unique()

In [ ]:
train_df['srcid_tier'].unique()

In [ ]:
dfs = [train_df, test_df]

for df in dfs:
    
    df['doj'] = pd.to_datetime(df['doj'], format = '%Y-%m-%d')

    # Extracting new columns with date columns
    df['doj' + '_year'] = df['doj'].dt.year
    df['doj' + '_month'] = df['doj'].dt.month
    df['doj' + '_day'] = df['doj'].dt.day
    df['doj' + '_dayofweek'] = df['doj'].dt.dayofweek
    df['doj' + '_isweekend'] = df['doj'].dt.dayofweek.isin([5,6]).astype(int)

    # Deleting the datetime datatype columns
    df.drop(['doj'], axis = 1, inplace = True)

##### info() of dataframes

In [ ]:
for i in dfs:
    print(i.info())

##### Check for Null values

In [ ]:
for i in dfs:   
    print(i.isna().sum(),'\n')

###### Insight
- There are no Nulll values

##### Check for duplicates

In [ ]:
for i in dfs:
    print(i.duplicated().sum())

###### Insights 
- No Duplicates

#### Coverting objects into Date

In [ ]:
date_item_list = ['doj', 'doi']

for df in dfs:
    for col in date_item_list:
        if col in df.columns and df[col].dtype != 'datetime64[ns]':
            df[col] = pd.to_datetime(df[col], format = '%Y-%m-%d')

            # Extracting new columns with date columns
            df[col + '_year'] = df[col].dt.year
            df[col + '_month'] = df[col].dt.month
            df[col + '_day'] = df[col].dt.day
            df[col + '_dayofweek'] = df[col].dt.dayofweek
            df[col + '_isweekend'] = df[col].dt.dayofweek.isin([5,6]).astype(int)

            # Deleting the datetime datatype columns
            df.drop([col], axis = 1, inplace = True)

## Preprocessing

In [ ]:
categorical_columns = [col for col in train_df.columns if train_df[col].dtype == 'O']
# ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']

In [ ]:
encoder = OrdinalEncoder(dtype = int)

train_df[categorical_columns] = encoder.fit_transform(train_df[categorical_columns])
test_df[categorical_columns] = encoder.fit_transform(test_df[categorical_columns])

## Data Preparation

In [ ]:
X = train_df.drop(['final_seatcount'], axis = 1)
y = train_df['final_seatcount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
print(train_df.columns)
print(test_df.columns)

### model testing

In [ ]:
model = XGBRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred)) # 504.88480654785326
rmse